In [1]:
import pandas as pd

# OBD2 + Dashcommand

## Limpeza dos dados adiquiridos por um Volkswagen UP 2014, em trecho urbano

### OBD2?

No meu caso utilizei o V-Gate comprado no [Aliexpress](https://pt.aliexpress.com/item/2015-promotion-Vgate-iCar2-Bluetooth-OBD-Scanner-iCar-2-elm327-Bluetooth-Diagnostic-Interface-with-Free-Shipping/32383340743.html?spm=a2g03.search0104.3.18.6cd43a947zdVsD&ws_ab_test=searchweb0_0,searchweb201602_6_10065_10068_10547_319_10059_10884_317_10548_10887_10696_321_322_10084_453_10083_454_10103_10618_10307_537_536_10902,searchweb201603_51,ppcSwitch_0&algo_expid=f18fc69d-e8ee-4a82-8d5e-7b05092fa4df-2&algo_pvid=f18fc69d-e8ee-4a82-8d5e-7b05092fa4df&transAbTest=ae803_4) com conexão wi-fi, você facilmente encontra também com conexão Bluetooth

![](https://images-na.ssl-images-amazon.com/images/I/71xAWWk0XyL._SY355_.jpg)


### Software

Utilizei o [Dashcommand](https://www.palmerperformance.com/products/dashcommand/), que é pago, mas você pode experimentar por 14 dias grátis
![](https://lh5.ggpht.com/p_Kiem3w8oNU1_MnMOvLrRgkjvIZBjMrQ3vFBn4Zgb0B-GGBqQUCLHTUtWicV9h-ivz3)


## <span style="color:red">Importante, a quantidade de colunas no seu Dataframe depende diretamente dos PIDs configurados no software</span>

### importando o arquivo CSV para um Dataframe

In [2]:
csv = 'csv/20200504.csv'

In [3]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_seq_items',100)
df = pd.read_csv(csv, low_memory=False, skiprows=0)
df.shape

(14714, 247)

### Vamos ter uma ideia do que temos em nosso Dataframe
Utilizado o comando df.head() para ter uma visão geral do arquivo importado, nele conseguimos observar diversas colunas com a informação de 'timestamp', o qual iremos descartar

atualmente o dataframe possui 251 colunas

In [4]:
df.head()


,Frame Number,Frame Time (h:m:s.ms),Frame Time (ms),Frame Time (delta),AUX.ACCEL.FORWARD (timestamp),AUX.ACCEL.FORWARD Gs,AUX.ACCEL.LATERAL (timestamp),AUX.ACCEL.LATERAL Gs,AUX.ROTATION.ROLL (timestamp),AUX.ROTATION.ROLL °,AUX.ROTATION.PITCH (timestamp),AUX.ROTATION.PITCH °,AUX.GPS.ALTITUDE (timestamp),AUX.GPS.ALTITUDE ft,AUX.GPS.ALTITUDE m,AUX.GPS.HORZ_ACCURACY (timestamp),AUX.GPS.HORZ_ACCURACY ft,AUX.GPS.HORZ_ACCURACY m,AUX.GPS.VERT_ACCURACY (timestamp),AUX.GPS.VERT_ACCURACY ft,AUX.GPS.VERT_ACCURACY m,AUX.GPS.COURSE (timestamp),AUX.GPS.COURSE °,AUX.GPS.SPEED (timestamp),AUX.GPS.SPEED mph,AUX.GPS.SPEED km/h,SAE.ECT (timestamp),SAE.ECT °F,SAE.ECT °C,SAE.SHRTFT1 (timestamp),SAE.SHRTFT1 %,SAE.LONGFT1 (timestamp),SAE.LONGFT1 %,SAE.MAP (timestamp),SAE.MAP inHg,SAE.MAP kPa,SAE.RPM (timestamp),SAE.RPM rpm,SAE.VSS (timestamp),SAE.VSS mph,SAE.VSS km/h,SAE.SPARKADV (timestamp),SAE.SPARKADV °,SAE.IAT (timestamp),SAE.IAT °F,SAE.IAT °C,SAE.TP (timestamp),SAE.TP %,SAE.MIL_DIST (timestamp),SAE.MIL_DIST miles,SAE.MIL_DIST kms,SAE.EVAP_PCT (timestamp),SAE.EVAP_PCT %,SAE.FLI (timestamp),SAE.FLI %,SAE.WARM_UPS (timestamp),SAE.WARM_UPS counts,SAE.BARO (timestamp),SAE.BARO inHg,SAE.BARO kPa,SAE.CATEMP11 (timestamp),SAE.CATEMP11 °F,SAE.CATEMP11 °C,SAE.VPWR (timestamp),SAE.VPWR V,SAE.LOAD_ABS (timestamp),SAE.LOAD_ABS %,SAE.LAMBDA (timestamp),SAE.LAMBDA lambda,SAE.TP_R (timestamp),SAE.TP_R %,SAE.AAT (timestamp),SAE.AAT °F,SAE.AAT °C,SAE.ALCH_PCT (timestamp),SAE.ALCH_PCT %,CONST.MAX_RPM (timestamp),CONST.MAX_RPM rpm,CONST.MAX_RPM r/min,CALC.TRIP (timestamp),CALC.TRIP,CALC.FUEL_CONSUMPTION (timestamp),CALC.FUEL_CONSUMPTION,CONST.LAMBDA_COMMANDED_PID (timestamp),CONST.LAMBDA_COMMANDED_PID,CONST.FLI_PID (timestamp),CONST.FLI_PID,CONST.MAF_PID (timestamp),CONST.MAF_PID,CONST.MAP_PID (timestamp),CONST.MAP_PID,CONST.STOICHIOMETRIC_AIR_FUEL_RATIO (timestamp),CONST.STOICHIOMETRIC_AIR_FUEL_RATIO :1,CONST.FUEL_DENSITY (timestamp),CONST.FUEL_DENSITY lb/gal (US) | lb/gal (UK),CONST.FUEL_DENSITY g/l,CONST.SPEED_CORRECTION (timestamp),CONST.SPEED_CORRECTION %,CONST.SPEED_CORRECTION coefficient,CONST.ENGINE_DISPLACEMENT (timestamp),CONST.ENGINE_DISPLACEMENT in³,CONST.ENGINE_DISPLACEMENT l,CONST.WHEEL_CIRCUMFERENCE (timestamp),CONST.WHEEL_CIRCUMFERENCE r/mile,CONST.WHEEL_CIRCUMFERENCE r/km,CONST.FINAL_DRIVE_RATIOS (timestamp),CONST.FINAL_DRIVE_RATIOS,CONST.MIN_RPM (timestamp),CONST.MIN_RPM rpm,CONST.MIN_RPM r/min,CONST.FUEL_CO2_CONTENT (timestamp),CONST.FUEL_CO2_CONTENT lb/gal (US) | lb/gal (UK),CONST.FUEL_CO2_CONTENT kg/l,CONST.LAMBDA_ACTUAL_PID (timestamp),CONST.LAMBDA_ACTUAL_PID,CONST.FUEL_TANK_CAPACITY (timestamp),CONST.FUEL_TANK_CAPACITY gal (US) | gal (UK),CONST.FUEL_TANK_CAPACITY l,CONST.SHIFT_RPM (timestamp),CONST.SHIFT_RPM rpm,CONST.SHIFT_RPM r/min,CALC.AIR_DENSITY (timestamp),CALC.AIR_DENSITY lb/ft³,CALC.AIR_DENSITY kg/m³,CALC.MAF_A (timestamp),CALC.MAF_A lb/min,CALC.MAF_A g/s,CALC.IS_NEW_DAY (timestamp),CALC.IS_NEW_DAY true | false,CONST.SPEED_ZONES (timestamp),CONST.SPEED_ZONES mph,CONST.SPEED_ZONES km/h,CALC.MAF (timestamp),CALC.MAF lb/min,CALC.MAF g/s,CALC.FLI (timestamp),CALC.FLI %,CALC.LAMBDA_COMMANDED (timestamp),CALC.LAMBDA_COMMANDED lambda,CALC.LAMBDA_ACTUAL (timestamp),CALC.LAMBDA_ACTUAL lambda,CALC.AFR_COMMANDED (timestamp),CALC.AFR_COMMANDED :1,CALC.MAP (timestamp),CALC.MAP inHg,CALC.MAP kPa,CALC.FUEL_FLOW (timestamp),CALC.FUEL_FLOW gal (US)/h | gal (UK)/h,CALC.FUEL_FLOW l/h,CALC.VSS_C (timestamp),CALC.VSS_C mph,CALC.VSS_C km/h,CALC.BOOST_PRESSURE (timestamp),CALC.BOOST_PRESSURE psi,CALC.BOOST_PRESSURE kPa | Bar | kg-f/cm²,CALC.TRIP.STOPS (timestamp),CALC.TRIP.STOPS,CALC.DISTANCE (timestamp),CALC.DISTANCE miles,CALC.DISTANCE km,CALC.FC.FUEL_VOLUME (timestamp),CALC.FC.FUEL_VOLUME gal (US) | gal (UK),CALC.FC.FUEL_VOLUME l,CALC.FC.AFC (timestamp),CALC.FC.AFC mpg (US) | mpg (UK),CALC.FC.AFC kml,CALC.FUEL_FLOW_AVG (timestamp),CALC.FUEL_FLOW_AVG gal (US)/h | gal (UK)/h,CALC.FUEL_FLOW_AVG l/h,CALC.TRIP.FUEL_FLOW (timestamp),CALC.TRIP.FUEL_FLOW gal (US

## Coluna timestamp
Como podemos ver as colunas timestamp são inuteis em nosso dataset, sempre se repetindo contando os 'ms'

O procedimento abaixo é para remover as colunas 'timestamp'

Poderia usar as o comando
> ***filter(like='timestamp')***

No entanto todo o dataset ficaria como objeto, por este motivo optei pelo metodo abaixo (talvez não seja o mais simples)





Explicado o motivo, vamos aos procedimentos


1. Criar a variável rtstamp contendo somente as colunas que apresentam as palavras 'timestamp' na descrição através do comando filter e salvá-lo em um novo csv

In [5]:
rtstamp = df.filter(like='timestamp')
rtstamp.to_csv('temp/rtstamp.csv', index=False)

Tudo isso para poder re-importar pulando a primeira linha do csv em um dataframe temporário (**dftemp**) e eliminando o Index das colunas

*Eu sei, deve ter algum meio mais fácil... mas vamos em frente*

In [6]:
dftemp = pd.read_csv('temp/rtstamp.csv', skiprows=1).columns

Vamos deixar nosso dataframe temporário de molho por um tempo

![](https://media.giphy.com/media/4gdveTpNzr9ok/giphy.gif)


Vamos importar novamente o CSV original, mas desta vez pulando a primeira linha (onde tinhamos as referências do nome timestamp) e vamos olhar para ter uma visão de como está

In [7]:
df2 = pd.read_csv(csv, low_memory=False, skiprows=1)

In [8]:
df2.head()

,Frame Number,Frame Time (h:m:s.ms),Frame Time (ms),Frame Time (delta),Forward Acceleration,Forward Acceleration.1,Lateral Acceleration,Lateral Acceleration.1,Vehicle Roll,Vehicle Roll.1,Vehicle Pitch,Vehicle Pitch.1,GPS Altitude,GPS Altitude.1,GPS Altitude.2,GPS Horizontal Accuracy,GPS Horizontal Accuracy.1,GPS Horizontal Accuracy.2,GPS Vertical Accuracy,GPS Vertical Accuracy.1,GPS Vertical Accuracy.2,GPS Course (Direction of Travel),GPS Course (Direction of Travel).1,GPS Vehicle Speed,GPS Vehicle Speed.1,GPS Vehicle Speed.2,Engine Coolant Temperature,Engine Coolant Temperature.1,Engine Coolant Temperature.2,Short Term Fuel Trim - Bank 1,Short Term Fuel Trim - Bank 1.1,Long Term Fuel Trim - Bank 1,Long Term Fuel Trim - Bank 1.1,Intake Manifold Absolute Pressure,Intake Manifold Absolute Pressure.1,Intake Manifold Absolute Pressure.2,Engine RPM,Engine RPM.1,Vehicle Speed Sensor,Vehicle Speed Sensor.1,Vehicle Speed Sensor.2,Ignition Timing Advance for #1 Cylinder,Ignition Timing Advance for #1 Cylinder.1,Intake Air Temperature,Intake Air Temperature.1,Intake Air Temperature.2,Absolute Throttle Position,Absolute Throttle Position.1,Distance Travelled While MIL is Activated,Distance Travelled While MIL is Activated.1,Distance Travelled While MIL is Activated.2,Commanded Evaporative Purge,Commanded Evaporative Purge.1,Fuel Level Input,Fuel Level Input.1,Number of warm-ups since diagnostic trouble codes cleared,Number of warm-ups since diagnostic trouble codes cleared.1,Barometric Pressure,Barometric Pressure.1,Barometric Pressure.2,"Catalyst Temperature Bank 1, Sensor 1","Catalyst Temperature Bank 1, Sensor 1.1","Catalyst Temperature Bank 1, Sensor 1.2",Control module voltage,Control module voltage.1,Absolute Load Value,Absolute Load Value.1,Fuel/Air Commanded Equivalence Ratio,Fuel/Air Commanded Equivalence Ratio.1,Relative Throttle Position,Relative Throttle Position.1,Ambient Air Temperature,Ambient Air Temperature.1,Ambient Air Temperature.2,Alcohol Fuel Percentage,Alcohol Fuel Percentage.1,Maximum engine speed validated from the 'Maximum engine speed' vehicle setting,Maximum engine speed validated from the 'Maximum engine speed' vehicle setting.1,Maximum engine speed validated from the 'Maximum engine speed' vehicle setting.2,Trip computer calculations,Trip computer calculations.1,Fuel economy/consumption calculations,Fuel economy/consumption calculations.1,Commanded lambda PID used by the system,Commanded lambda PID used by the system.1,Fuel level input PID used by the system,Fuel level input PID used by the system.1,Mass air flow PID used by the system,Mass air flow PID used by the system.1,Manifold absolute pressure PID used by the system,Manifold absolute pressure PID used by the system.1,Stoichiometric air/fuel ratio derived from the 'Fuel type' vehicle setting,Stoichiometric air/fuel ratio derived from the 'Fuel type' vehicle setting.1,Fuel density derived from the 'Fuel type' vehicle setting,Fuel density derived from the 'Fuel type' vehicle setting.1,Fuel density derived from the 'Fuel type' vehicle setting.2,Speed correction coefficient validated and converted from the 'Vehicle speed correction factor' vehicle setting,Speed correction coefficient validated and converted from the 'Vehicle speed correction factor' vehicle setting.1,Speed correction coefficient validated and converted from the 'Vehicle speed correction factor' vehicle setting.2,Engine displacement derived from 'Engine displacement' vehicle setting,Engine displacement derived from 'Engine displacement' vehicle setting.1,Engine displacement derived from 'Engine displacement' vehicle setting.2,Wheel circumference derived from vehicle settings,Wheel circumference derived from vehicle settings.1,Wheel circumference derived from vehicle settings.2,Final drive ratios validated and converted from 'Final drive ratios' vehicle setting,Final drive ratios validated and converted from 'Final drive ratios' vehicle setting.1,Minimum engine speed validated from the 'Mini

Hora de resgatar a nossa **dftemp** que estava de molho e usá-la para o motivo que a criamos: Limpar nosso dataframe


In [10]:
df2.drop(dftemp, axis=1, inplace = True)

In [11]:
df2.head()
print(df2.shape)

(14713, 152)


Todos os dados das colunas "trip" ou "Trip" não nos são uteis, portanto vamos removê-las

In [12]:
rip = df2.filter(regex='rip').columns
df2.drop(rip,axis=1, inplace = True)


Vamos olhar para as features de string, ver se tem algum útil em alguma delas

In [13]:
df2.select_dtypes(include=['object']).tail(10)

,Frame Time (h:m:s.ms),Fuel economy/consumption calculations.1,Commanded lambda PID used by the system.1,Fuel level input PID used by the system.1,Mass air flow PID used by the system.1,Manifold absolute pressure PID used by the system.1,Final drive ratios validated and converted from 'Final drive ratios' vehicle setting.1,Returns true when a new day starts.1,Average fuel economy/consumption for past periods.1,Average fuel economy/consumption for past periods.2,"Current and suggested gear with shift indicator, calculated based on speed and engine rpm.1","Current and suggested gear with shift indicator, calculated based on speed and engine rpm.2",Manage values displyed on Fillup dashboard.1
14703,0:31:11.587,Active,SAE.LAMBDA,SAE.FLI,CALC.MAF_A,SAE.MAP,"18.58,10.33,6.31,4.57,3.65",False,"40.100000,42.900000,29.100000,21.700000,16.300...","7.100000,6.600000,9.800000,13.200000,17.700000...",NaN,"0,0,0,5","6.251000,0.000000,3.50"
14704,0:31:11.708,Active,SAE.LAMBDA,SAE.FLI,CALC.MAF_A,SAE.MAP,"18.58,10.33,6.31,4.57,3.65",False,"40.100000,42.900000,29.100000,21.700000,16.300...","7.100000,6.600000,9.800000,13.200000,17.700000...",NaN,"0,0,0,5","6.251000,0.000000,3.50"
14705,0:31:11.834,Active,SAE.LAMBDA,SAE.FLI,CALC.MAF_A,SAE.MAP,"18.58,10.33,6.31,4.57,3.65",False,"40.100000,42.900000,29.100000,21.700000,16.300...","7.100000,6.600000,9.800000,13.200000,17.700000...",NaN,"0,0,0,5","6.251000,0.000000,3.50"
14706,0:31:11.863,Active,SAE.LAMBDA,SAE.FLI,CALC.MAF_A,SAE.MAP,"18.58,10.33,6.31,4.57,3.65",False,"40.100000,42.900000,29.100000,21.700000,16.300...","7.100000,6.600000,9.800000,13.200000,17.700000...",NaN,"0,0,0,5","6.251000,0.000000,3.50"
14707,0:31:11.880,Active,SAE.LAMBDA,SAE.FLI,CALC.MAF_A,SAE.MAP,"18.58,10.33,6.31,4.57,3.65",False,"40.100000,42.900000,29.100000,21.700000,16.300...","7.100000,6.600000,9.800000,13.200000,17.700000...",NaN,"0,0,0,5","6.251000,0.000000,3.50"
14708,0:31:12.024,Active,SAE.LAMBDA,SAE.FLI,CALC.MAF_A,SAE.MAP,"18.58,10.33,6.31,4.57,3.65",False,"40.100000,42.900000,29.100000,21.700000,16.300...","7.100000,6.600000,9.800000,13.200000,17.700000...",NaN,"0,0,0,5","6.251000,0.000000,3.50"
14709,0:31:12.123,Active,SAE.LAMBDA,SAE.FLI,CALC.MAF_A,SAE.MAP,"18.58,10.33,6.31,4.57,3.65",False,"40.100000,42.900000,29.100000,21.700000,16.300...","7.100000,6.600000,9.800000,13.200000,17.700000...",NaN,"0,0,0,5","6.251000,0.000000,3.50"
14710,0:31:12.204,Active,SAE.LAMBDA,SAE.FLI,CALC.MAF_A,SAE.MAP,"18.58,10.33,6.31,4.57,3.65",False,"40.100000,42.900000,29.100000,21.700000,16.300...","7.100000,6.600000,9.800000,13.200000,17.700000...",NaN,"0,0,0,5","6.251000,0.000000,3.50"
14711,0:31:12.289,Active,SAE.LAMBDA,SAE.FLI,CALC.MAF_A,SAE.MAP,"18.58,10.33,6.31,4.57,3.65",False,"40.100000,42.900000,29.100000,21.700000,16.300...","7.100000,6.600000,9.800000,13.200000,17.700000...",NaN,"0,0,0,5","6.251000,0.000000,3.50"
14712,0:31:12.370,Active,SAE.LAMBDA,SAE.FLI,CALC.MAF_A,SAE.MAP,"18.58,10.33,6.31,4.57,3.65",False,"40.100000,42.900000,29.100000,21.700000,16.300...","7.100000,6.600000,9.800000,13.200000,17.700000...",NaN,"0,0,0,5","6.251000,0.000000,3.50"


Não, podemos ignorá-las e utilizar somente as features númericas
Selecionar somente as colunas com valor inteiros e flutuantes

In [14]:
df2 = df2.select_dtypes(include=['float64', 'int64'])
print(df2.shape)

(14713, 112)


In [15]:
df2 = df2.dropna(axis=1, how='all') #para deletar todas as colunas onde todo os valores são NaN
df2.shape


(14713, 102)

As colunas com "price" e "reset" também podem ser descartadas, vamos apenas checar antes de deletá-las

In [16]:
pricecol = df2.filter(like=('price')).columns.to_list()
resetcol = df2.filter(like=('reset')).columns.to_list()
df2[pricecol+resetcol]

,Fuel price entered at last fillup.1,Fuel price entered at last fillup.2,Distance travelled since last fuel consumption reset.1,Distance travelled since last fuel consumption reset.2,Volume of fuel consumed since last fuel consumption reset.1,Volume of fuel consumed since last fuel consumption reset.2,Average fuel economy/consumption since last fuel consumption reset.2,Average fuel economy/consumption since last fuel consumption reset.3,Average fuel economy/consumption since last fuel consumption reset.4,Average fuel economy/consumption since last fuel consumption reset.5
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
14708,3.5,3.5,212.9,342.6,6.2,28.4,18.3,1250584.0,34.1,12.1
14709,3.5,3.5,212.9,342.6,6.2,28.4,18.3,1250584.0,34.1,12.1
14710,3.5,3.5,212.9,342.6,6.2,28.4,18.3,1250584.0,34.1,12.1
14711,3.5,3.5,212.9,342.6,6.2,28.4,18.3,1250584.0,34.1,12.1


In [17]:
deletar= ['Distance Travelled While MIL is Activated.1',
 'Distance Travelled While MIL is Activated.2',
 'Number of warm-ups since diagnostic trouble codes cleared.1',
 "Maximum engine speed validated from the 'Maximum engine speed' vehicle setting.2",
 "Minimum engine speed validated from the 'Minimum engine speed' vehicle setting.2",
 'Estimated mass air flow - method A.1',
 'Estimated mass air flow - method A.2',
 'Fuel consumption correction factor based on fillup adjustment.1',
 'Fuel consumption correction factor based on fillup adjustment.2']

In [18]:
df2.drop(pricecol+resetcol+deletar, axis=1, inplace = True)

Agora vamos ver todas as colunas do dataset para poder jogar em um dicionário e renomeá-las conforme o nosso gosto

In [19]:
df2.columns.to_list()

['Frame Number',
 'Frame Time (ms)',
 'Frame Time (delta)',
 'Forward Acceleration.1',
 'Lateral Acceleration.1',
 'Vehicle Roll.1',
 'Vehicle Pitch.1',
 'GPS Altitude.1',
 'GPS Altitude.2',
 'GPS Horizontal Accuracy.1',
 'GPS Horizontal Accuracy.2',
 'GPS Vertical Accuracy.1',
 'GPS Vertical Accuracy.2',
 'GPS Course (Direction of Travel).1',
 'GPS Vehicle Speed.1',
 'GPS Vehicle Speed.2',
 'Engine Coolant Temperature.1',
 'Engine Coolant Temperature.2',
 'Short Term Fuel Trim - Bank 1.1',
 'Long Term Fuel Trim - Bank 1.1',
 'Intake Manifold Absolute Pressure.1',
 'Intake Manifold Absolute Pressure.2',
 'Engine RPM.1',
 'Vehicle Speed Sensor.1',
 'Vehicle Speed Sensor.2',
 'Ignition Timing Advance for #1 Cylinder.1',
 'Intake Air Temperature.1',
 'Intake Air Temperature.2',
 'Absolute Throttle Position.1',
 'Commanded Evaporative Purge.1',
 'Barometric Pressure.1',
 'Barometric Pressure.2',
 'Catalyst Temperature Bank 1, Sensor 1.1',
 'Catalyst Temperature Bank 1, Sensor 1.2',
 'Contr

In [20]:
novascols = {'Frame Number':'Frame Number', 'Frame Time (ms)':'Frame Time (ms)', 'Frame Time (delta)':'Frame Time (delta)','Forward Acceleration.1' : 'Forward acc (g)','Lateral Acceleration.1':'Lateral acc (g)', 'Vehicle Roll.1':'Roll',
       'Vehicle Pitch.1':'Pitch', 'GPS Latitude.1':'Latitude', 'GPS Longitude.1':'Longitude',
       'GPS Altitude.1':'Altitude (ft)', 'GPS Altitude.2':'Altitude (m)', 'GPS Horizontal Accuracy.1':'GPS Hor Accuracy (ft)',
       'GPS Horizontal Accuracy.2':'GPS Hor Accuracy (m)', 'GPS Vertical Accuracy.1':'GPS Vert Accuracy (ft)',
       'GPS Vertical Accuracy.2':'GPS Vert Accuracy (m)', 'GPS Course (Direction of Travel).1':'GPS Direction',
       'GPS Vehicle Speed.1': 'GPS Speed (mph)', 'GPS Vehicle Speed.2':'GPS Speed (km/h)',
       'Engine Coolant Temperature.1':'Engine Coolant (F)', 'Engine Coolant Temperature.2':'Engine Coolant (C)','Intake Manifold Absolute Pressure.1':'Intake Abs Pressure (inHg)',
       'Intake Manifold Absolute Pressure.2':'Intake Abs Pressure (kPa)', 'Engine RPM.1':'Engine RPM',
       'Vehicle Speed Sensor.1':'Speed Sensor (mph)', 'Vehicle Speed Sensor.2':'Speed Sensor (km/h)',
       'Ignition Timing Advance for #1 Cylinder.1':'Ignition Timing Advance', 'Intake Air Temperature.1':'Intake Air Temperature (F)',
       'Intake Air Temperature.2':'Intake Air Temperature (C)','Number of warm-ups since diagnostic trouble codes cleared.1':'Warms-ups',
       'Barometric Pressure.1':'Barometric Pressure (inHg)', 'Barometric Pressure.2':'Barometric Pressure (kPA)',
       'Catalyst Temperature Bank 1, Sensor 1.1':'Catalyst Temp (F)',
       'Catalyst Temperature Bank 1, Sensor 1.2':'Catalyst Temp (C)', 'Absolute Load Value.1':'Load (%)',
       'Fuel/Air Commanded Equivalence Ratio.1':'Lambda', 'Ambient Air Temperature.1':'Ambient Air Temp (F)',
       'Ambient Air Temperature.2':'Ambient Air Temp (C)', 'Alcohol Fuel Percentage.1':'Alcohol Percentage',
       'Wheel circumference derived from vehicle settings.1':'Wheel Circumference (r/mile)',
       'Wheel circumference derived from vehicle settings.2':'Wheel Circumference (r/km)',"Engine displacement derived from 'Engine displacement' vehicle setting.1":'Engine displacement (in3)',
       "Engine displacement derived from 'Engine displacement' vehicle setting.2":'Engine displacement (L)',
       "Fuel density derived from the 'Fuel type' vehicle setting.1":'Fuel density (lb/gal)',
       "Fuel density derived from the 'Fuel type' vehicle setting.2":'Fuel density (g/L)',
       "Stoichiometric air/fuel ratio derived from the 'Fuel type' vehicle setting.1":'Air/Fuel Ratio',
       'Fuel tank capacity from vehicle setting.1':'Tank Capacity (gal)',
       'Fuel tank capacity from vehicle setting.2':'Tank Capacity (L)',
       "Maximum engine speed validated from the 'Maximum engine speed' vehicle setting.1":'Max RPM',
       "Minimum engine speed validated from the 'Minimum engine speed' vehicle setting.1":'Min RPM', 'Ambient air density.1':'Ambient air density (lb/ft2)',
       'Ambient air density.2':'Ambient air density (Kg/m2)','Mass air flow - either uses the specified PID from vehicle settings or the best for the vehicle if no setting is provided.1':'MAF PID(lb/min)',
       'Mass air flow - either uses the specified PID from vehicle settings or the best for the vehicle if no setting is provided.2':'MAF PID(g/s)',
       'Fuel Level Input.3':'Fuel Level (%)',
       "Commanded lambda derived from 'Lambda (commanded) PID' vehicle setting or the best for the vehicle if no setting is provided.1":'Commanded Lambda',
       'Fuel flow calculated based on a MAF value (estimated or actual).1':'Fuel Flow (gal/h)',
       'Fuel flow calculated based on a MAF value (estimated or actual).2':'Fuel Flow (L/h)',
       'Corrected vehicle speed.1':'Corrected speed (mph)', 'Corrected vehicle speed.2':'Corrected speed (km/h)',
       'Manifold absolute pressure - either uses SAE.MAP or the best calculation for the vehicle if SAE.MAP is not provided.1':'Manifold absolute pressure (inHg)',
       'Manifold absolute pressure - either uses SAE.MAP or the best calculation for the vehicle if SAE.MAP is not provided.2':'Manifold absolute pressure (kPa)',
       'Vacuum estimation.1':'Vacuum (inHg)', 'Vacuum estimation.2':'Vacuum (kPa)',
       'Instantaneous fuel consumption.1':'Inst. fuel consumption (mpg)', 'Instantaneous fuel consumption.2':'Inst. fuel consumption (km/L)',
       'Boost pressure estimation.1':'Boost pressure estimation (psi)', 'Boost pressure estimation.2':'Boost pressure estimation (kPa)',
       'Current acceleration based on the last two speed readings.1':'Current acceleration (ft/s2)',
       'Current acceleration based on the last two speed readings.2':'Current acceleration (m/s2)',
       'Average fuel flow.1':'Average fuel flow (gal/h)', 'Average fuel flow.2':'Average fuel flow (L/h)',
       'Calculated engine power.1':'Engine Power (hp)',
       'Currently engaged gear.1':'Currently engaged gear', 'Acceleration expressed as g (gravity).1':'Acceleration as G force',
       'Calculated engine torque.1':'Engine torque (lb*ft)', 'Calculated engine torque.2':'Engine torque (kgf*m)',
       'Vehicle Voltage Level (usually the input voltage at the diagnostic connector).1':'Voltage',
       'Absolute Throttle Position.1':'Absolute Throttle Position (%)', 'Accelerator Pedal Position D.1':'Accelerator Pedal Position D (%)',
       'Short Term Fuel Trim - Bank 1.1':'S_Term Fuel Trim (%)','Long Term Fuel Trim - Bank 1.1':'L_Term Fuel Trim (%)','Commanded Evaporative Purge.1':'Commanded Ev. Purge',
       'Control module voltage.1':'Module Voltage (V)', 'Relative Throttle Position.1':'Relative Throttle Position (%)'}

In [21]:
df2.rename(novascols, axis='columns' ,inplace=True)
df2.shape

(14713, 83)

In [22]:
df2.columns.to_list() #listar novamente as colunas para ter certeza que nenhuma ficou esquecida e não foi renomeada

['Frame Number',
 'Frame Time (ms)',
 'Frame Time (delta)',
 'Forward acc (g)',
 'Lateral acc (g)',
 'Roll',
 'Pitch',
 'Altitude (ft)',
 'Altitude (m)',
 'GPS Hor Accuracy (ft)',
 'GPS Hor Accuracy (m)',
 'GPS Vert Accuracy (ft)',
 'GPS Vert Accuracy (m)',
 'GPS Direction',
 'GPS Speed (mph)',
 'GPS Speed (km/h)',
 'Engine Coolant (F)',
 'Engine Coolant (C)',
 'S_Term Fuel Trim (%)',
 'L_Term Fuel Trim (%)',
 'Intake Abs Pressure (inHg)',
 'Intake Abs Pressure (kPa)',
 'Engine RPM',
 'Speed Sensor (mph)',
 'Speed Sensor (km/h)',
 'Ignition Timing Advance',
 'Intake Air Temperature (F)',
 'Intake Air Temperature (C)',
 'Absolute Throttle Position (%)',
 'Commanded Ev. Purge',
 'Barometric Pressure (inHg)',
 'Barometric Pressure (kPA)',
 'Catalyst Temp (F)',
 'Catalyst Temp (C)',
 'Module Voltage (V)',
 'Load (%)',
 'Lambda',
 'Relative Throttle Position (%)',
 'Ambient Air Temp (F)',
 'Ambient Air Temp (C)',
 'Alcohol Percentage',
 'Max RPM',
 'Air/Fuel Ratio',
 'Fuel density (lb/gal)'

In [23]:
name = csv[4:-4]

In [24]:
df2.to_csv('csvcleaned/'+name+'.csv', index=False)


